[<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/snntorch_alpha_w.png?raw=true' width="300">](https://github.com/jeshraghian/snntorch/)

# snnMetrics Tutorial
### Tutorial written by Sahil Konjarla

<a href="https://colab.research.google.com/github/jeshraghian/snntorch/blob/master/examples/quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

[<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/GitHub-Mark-Light-120px-plus.png?raw=true' width="28">](https://github.com/ruhai-lin/SNN-Exoplanet-Hunter) [<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/GitHub_Logo_White.png?raw=true' width="80">](https://github.com/ruhai-lin/SNN-Exoplanet-Hunter)


For a comprehensive overview on how SNNs work, and what is going on under the hood, [then you might be interested in the snnTorch tutorial series available here.](https://snntorch.readthedocs.io/en/latest/tutorials/index.html)
The snnTorch tutorial series is based on the following paper. If you find these resources or code useful in your work, please consider citing the following source:


> <cite> [Jason K. Eshraghian, Max Ward, Emre Neftci, Xinxin Wang, Gregor Lenz, Girish Dwivedi, Mohammed Bennamoun, Doo Seok Jeong, and Wei D. Lu. "Training Spiking Neural Networks Using Lessons From Deep Learning". Proceedings of the IEEE, 111(9) September 2023.](https://ieeexplore.ieee.org/abstract/document/10242251) </cite>

# 0. Introduction

In this tutorial, you will learn how to combine snnTorch and snnMetrics to:
* calculate the number of synaptic operations in a single patch
* calculate the number of synaptic operations in a epoch

During this tutorial we will be using the original static MNIST dataset and train a multi-layer fully-connected spiking neural network and a multi-layer convolutional spiking neural network. Calculating the batch and epoch synaptic statistics for each network.




# 1. snnMetrics
Before we get into the bulk of this tutorial, it is beneficial to understand what exactly is snnMetrics and what are the benefits of using snnMetrics in your spikng neural network code.

##1.1 What is snnMetrics
snnMetrics is an open-source python library, made by Gregor Lenz, that can be used to provide metrics that are specific for spiking neural networks. The API is similar to [torchmetrics](https://lightning.ai/docs/torchmetrics/stable) and the package is currently in beta phase.


The current usage of snnMetrics allows users to track the number of spikes or synapic operations that occur in a single batch of training or during a single epoch. This tracking can allow us to understand the:
* Allows you to understand how the data you are passing through is affecting your network
* It can be used to do energy efficiency analysis, especially if you are using neuromorphic hardware.
* Can be used to understand network perfomance and optimization
* can provide insight into fault detection and help you diagnose any issues in your network
* Due to SNNs being adpet in processing temportal information, we can analyze spike patters and get insight into how the network handles time-dependent data
* It can also be used to track the number of firing neurons in your ANN layers

For more information on snnMetrics please check out the documentation page [here](https://snnmetrics.readthedocs.io/en/latest/index.html) or the [github page](https://github.com/open-neuromorphic/snnmetrics). Now let us get into how we can use snnMetrics.

# 2. snnMetrics Startup

First we need to initialize a new Git repository in our current directory, if you are using Google Colab if will just make a new directory in the runtime. Then instiall the Python Build Reasonablness package which helps simplify Python package movement. Furthermore, we need to install the snntorch python library and finally install snnmetrics.

In [2]:
!git init --quiet
!pip install pbr --quiet
!pip install snntorch --quiet
!pip install snnmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 5.5 MB/s eta 0:00:00


And then import the following libraries below.

In [3]:
# imports
import pbr # to manage the packages

# snntorch
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

# snnmetrics
import snnmetrics as sm

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 3. Setting up the Static MNIST Dataset

In [4]:
# Dataloader arguements
batch_size = 128
data_path='/data/mnist'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
# Define a transform
transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 115134368.42it/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 10527130.77it/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31976416.78it/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17131770.47it/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



If the above code blocks throws an error, e.g. the MNIST servers are down, then uncomment the following code instead.

In [6]:
# temporary dataloader if MNIST service is unavailable
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

# mnist_train = datasets.MNIST(root = './', train=True, download=True, transform=transform)
# mnist_test = datasets.MNIST(root = './', train=False, download=True, transform=transform)

In [7]:
# Create DataLoaders
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

# 4. Setting up Metric Computation

In [8]:
# Implementation of snnMetrics
# Calculating the number of synops per ANN and SNN layer
def metric_computation(ops_layer, spks):
    batch_stats_layer = ops_layer(spks)
    synops_per_neuron = batch_stats_layer['synops_per_neuron']
    synops = batch_stats_layer['synops']
    ops_layer.reset()
    return synops

In [9]:
# Printing the number of synaptic operations per batch
def synops_printer(fc1_synops,
        lif1_synops, fc2_synops,lif2_synops):
    total_synops_per_batch = fc1_synops + fc2_synops + lif1_synops + lif2_synops
    print(f"Synops for Linear Layer: {fc1_synops}")
    print(f"Synops for SNN Layer: {lif1_synops}")
    print(f"Synops for Linear layer: {fc2_synops}")
    print(f"Synops for SNN Layer: {lif2_synops}")
    print(f"Total Synops: {total_synops_per_batch}")
    print("\n")

We are defining a function that can calculate the metrics per batch. The inputs of our function are `ops_layer` and `spks`. `ops_layer` refers to the snnMetric layer that we add to our network, I will go into this more in the next section. `spks` refers to the spike tensor that is returned from adding a spiking layer to the network.

The actual computation is done by using:
* `batch_stats_layer` refers to the metrics layer where we pass in the spiking tensor `spks`
* `synops_per_neuron` attains the number of spikes per nueron
* `synops` gets the total number of synaptic operations that occured in that layer per each batch

At the end of our function we are reseting the `ops_layer` so that we have a clean layer.

# 5. Define a Fully-Connected SNN Network

In [10]:
# Network Architecture
num_inputs = 28*28
num_hidden = 1000
num_outputs = 10

# Temporal Dynamics
num_steps = 25
beta = 0.95

This block of code follows a similar syntax as the official [snnTorch tutorials](https://snntorch.readthedocs.io/en/latest/tutorials/index.html), but we are adding snnMetrics layers along with some other code.

In [11]:
# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.linear_layer1 = sm.SynOps(fanout=num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.snn_layer1 = sm.SynOps(fanout=num_hidden)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.linear_layer2 = sm.SynOps(fanout=num_outputs)
        self.lif2 = snn.Leaky(beta=beta)
        self.snn_layer2 = sm.SynOps(fanout=num_outputs)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        # Record the final layer
        spk2_rec = []
        mem2_rec = []

        # Define a threshold
        threshold_value = 0.0001

        for step in range(num_steps):
            # First Layer
            cur1 = self.fc1(x.flatten(1))
            thresholded_output = torch.where(cur1 > threshold_value, torch.tensor(1.0), torch.tensor(0.0))
            fc1_synops = metric_computation(self.linear_layer1, thresholded_output)

            # Second Layer
            spk1, mem1 = self.lif1(cur1, mem1)
            lif1_synops = metric_computation(self.snn_layer1, spk1)

            # Third Layer
            cur2 = self.fc2(spk1)
            thresholded_output2 = torch.where(cur2 > threshold_value, torch.tensor(1.0), torch.tensor(0.0))
            fc2_synops = metric_computation(self.linear_layer2, thresholded_output2)

            # Fourth Layer
            spk2, mem2 = self.lif2(cur2, mem2)
            lif2_synops = metric_computation(self.snn_layer2, spk2)

            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return fc1_synops, fc2_synops, lif1_synops, lif2_synops, torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

# Load the network onto CUDA if available
net = Net().to(device)

The definition of the network is very simlar to the tutorials, but we are adding snnMetrics layer inbetween every ANN and snntorch layer. They are defined as:
* `linear_layer1`
* `snn_layer1`
* `linear_layer2`
* `snn_layer2`

Here as you can see I defined each layer to be a snnMetrics layer `sm.SynOps()`. `sm.SynOps` takes in as an argument `fanout` which refers to the number of connections to the next layer. In this case we are refering it to be the our `num_hidden` and `num_outputs`.


In the `forward()` function I have defined a variable `threshold_value`. `threshold_value` was defined so that we could turn the tensor output from the linear transformation into spiking tensor. Using the Pytorch function `torch.where()`, which allowed for snnMetrics to be able to calculate the number of synaptic operations in a linear layer.

* `threshold_output` takes the output from the linear layer and converts it into a tensor of 0s and 1s.
* `fc` passes the snnMetrics layer defined and the spikes tensor into the `metric_computation` function that we have defined earlier
* `lif` pass the snnMetrics layer defind and the spikes tensor into the `metric_computation` function that we have defined earlier

Finally, the network returns all the synaptic operations for each layer so that our print function and give us those values for each test and train loop.

# 6. Training Fully-Connected SNN Network

## 6.1 Define the Loss function and the Optimizer

In [12]:
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999)) # Adam op[timizer with a learning rate of 5e^-4
loss = nn.CrossEntropyLoss() # Cross Entropy Loss function

## 6.2 Training and Testing model

We are going to be training for one epoch and print out the synaptic operations for each.

In [13]:
num_epochs = 1
loss_hist = []
test_loss_hist = []
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        fc1_synops, fc2_synops, lif1_synops, lif2_synops, spk2_rec, mem2_rec = net(data.view(batch_size, -1))

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            loss_val += loss(mem2_rec[step], targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Print Synops per batch
        print(f"Train Synops {counter}:")
        synops_printer(fc1_synops, lif1_synops, fc2_synops, lif2_synops)

        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device)
            test_targets = test_targets.to(device)

            # Test set forward pass
            test_fc1_synops, test_fc2_synops, test_lif1_synops, test_lif2_synops, test_spk2_rec, test_mem = net(test_data.view(batch_size, -1))

            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_mem[step], test_targets)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            print(f"Test Synops: {counter}")
            synops_printer(test_fc1_synops, test_lif1_synops, test_fc2_synops, test_lif2_synops)

            # Exit rule
            counter += 1
            if counter == 20:
                break

Train Synops 0:
Synops for Linear Layer: 498335.9375
Synops for SNN Layer: 52304.6875
Synops for Linear layer: 54.453125
Synops for SNN Layer: 1.953125
Total Synops: 550697.0


Test Synops: 0
Synops for Linear Layer: 496140.625
Synops for SNN Layer: 54328.125
Synops for Linear layer: 56.640625
Synops for SNN Layer: 1.953125
Total Synops: 550527.3125


Train Synops 1:
Synops for Linear Layer: 493812.5
Synops for SNN Layer: 53031.25
Synops for Linear layer: 54.453125
Synops for SNN Layer: 1.5625
Total Synops: 546899.75


Test Synops: 1
Synops for Linear Layer: 495656.25
Synops for SNN Layer: 56054.6875
Synops for Linear layer: 55.703125
Synops for SNN Layer: 5.234375
Total Synops: 551771.875


Train Synops 2:
Synops for Linear Layer: 497320.3125
Synops for SNN Layer: 53968.75
Synops for Linear layer: 55.625
Synops for SNN Layer: 2.96875
Total Synops: 551347.625


Test Synops: 2
Synops for Linear Layer: 501437.5
Synops for SNN Layer: 57281.25
Synops for Linear layer: 54.21875
Synops for S

# 7. Training a Convolutional Network

## 7.1 Define New Synops Print Function

In [ ]:
# Printing the number of synaptic operations per layer and batch
def synops_printer(conv1_synops, lif1_synops,
                   conv2_synops,lif2_synops, fc_synops, lif3_synops):
    total_synops_per_batch = conv1_synops + conv2_synops + lif1_synops + lif2_synops + fc_layer + lif3_synops
    print(f"Synops for Conv Layer: {conv1_synops}")
    print(f"Synops for SNN Layer: {lif1_synops}")
    print(f"Synops for Conv layer: {conv2_synops}")
    print(f"Synops for SNN Layer: {lif2_synops}")
    print(f"Synops for Linear Layer: {fc_synops}")
    print(f"Synops for SNN Layer: {lif3_synops}")
    print(f"Total Synops: {total_synops_per_batch}")
    print("\n")

##7.2 Define a Convolutional Network

This block of code follows a similar syntax to our fully-connected layer model and training, but instead we are going to be changing some aspects to fit the concolutional model.

In [ ]:
# Define Network
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.conv1 = nn.Conv2d(1, 24, 5, padding="same")
        self.conv1_layer = sm.SynOps(fanout=24)
        self.lif1 = snn.Leaky(beta=beta)
        self.lif1_layer = sm.SynOps(fanout=24)
        self.mp1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(24, 28, 5, padding="same")
        self.conv2_layer = sm.SynOps(fanout=7)
        self.lif2 = snn.Leaky(beta=beta)
        self.lif2_layer = sm.SynOps(fanout=7)
        self.mp2 = nn.MaxPool2d(2)

        self.fc = nn.Linear(28 * 7 * 7, 10)
        self.linear_layer = sm.SynOps(fanout=10)
        self.lif3 = snn.Leaky(beta=beta)
        self.lif3_layer = sm.SynOps(fanout=10)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []

        # Define a threshold
        threshold_value = 0.0001

        # time-loop
        for step in range(num_steps):
            # First Layer
            cur1 = self.conv1(x)
            thresholded_output = torch.where(cur1 > threshold_value, torch.tensor(1.0), torch.tensor(0.0))
            conv1_synops = metric_computation(self.conv1_layer, thresholded_output)
            spk1, mem1 = self.lif1(self.mp1(cur1), mem1)
            lif1_synops = metric_computation(self.lif1_layer, spk1)

            # Second Layer
            cur2 = self.conv2(spk1)
            thresholded_output2 = torch.where(cur2 > threshold_value, torch.tensor(1.0), torch.tensor(0.0))
            conv2_synops = metric_computation(self.conv2_layer, thresholded_output2)
            spk2, mem2 = self.lif2(self.mp2(cur2), mem2)
            lif2_synops = metric_computation(self.lif2_layer, spk2)

            # Third Layer
            cur3 = self.fc(spk2.flatten(1))
            thresholded_output3 = torch.where(cur3 > threshold_value, torch.tensor(1.0), torch.tensor(0.0))
            fc_synops = metric_computation(self.linear_layer, thresholded_output3)
            spk3, mem3 = self.lif3(cur3, mem3)
            lif3_synops = metric_computation(self.lif3_layer, spk3)

            spk3_rec.append(spk3)

        return conv1_synops, lif1_synops, conv2_synops, lif2_synops, fc_synops, lif3_synops, torch.stack(spk3_rec, dim=0)

# Load the network onto CUDA if available
convnet = ConvNet().to(device)

The definition of the convolutional network is very similar to fully-connected network we defined above, but we are adding convolutional layers instead of fully-connected layers and we are max pooling at the end. We defined synops layers:
* `conv1_layer`
* `lif1_layer`
* `conv2_layer`
* `lif2_layer`
* `linear_layer`
* `lif3_layer`

The snnMetrics syntax is all the same, but we are going to be chaning the fanout, similar to what we did for the fully-connected layers. Setting the fanouts to be respectivly, 24,7,10. The connections get smaller and smaller due to the fact that we are using maxpooling.


## 7.3 Define the Loss Function and Optimizer

In [ ]:
optimizer = torch.optim.Adam(convnet.parameters(), lr=5e-4, betas=(0.9, 0.999)) # Adam op[timizer with a learning rate of 5e^-4
loss = nn.CrossEntropyLoss() # Cross Entropy Loss function

## 7.4 Training Convolutional Network


In [ ]:
num_epochs = 1
loss_hist = []
test_loss_hist = []
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        spk3_rec = []
        convnet.train()
        conv1_synops, lif1_synops, conv2_synops, lif2_synops, fc_layer, lif3_synops, spk3_rec = convnet(data)

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        loss_val = loss(spk3_rec.sum(0), targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Print train/test loss/accuracy
        print("Train Synops")
        synops_printer(conv1_synops, lif1_synops, conv2_synops, lif2_synops, fc_layer, lif3_synops)

        with torch.no_grad():
            convnet.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device)
            test_targets = test_targets.to(device)

            # Test set forward pass
            test_conv1_synops, test_lif1_synops, test_conv2_synops, test_lif2_synops, test_fc_synops, test_lif3_synops, test_spk3_rec  = convnet(test_data.view(batch_size, 1, 28, 28))

            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_spk3_rec[step], test_targets)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            print(f"Test Synops:")
            synops_printer(conv1_synops, lif1_synops, conv2_synops, lif2_synops, fc_layer, lif3_synops)

            # Exit rule
            counter += 1
            if counter == 120:
                break

# Conclusion

Now after completing this tutorial you should know how to constuct and train fully-connected and convolutional networks on a static dataset. The snnMetrics library is still in its early stages, but it has so many applications and uses that will greatly benefit the neuromorphic computing community. Armed with this knowledge, you should now be able to perform different benchmarking and testing on your spiking neural networks.

# Additional Resources

* [Check out the snnMetric Github project here](https://github.com/open-neuromorphic/snnmetrics)
* [Check out the snnTorch Github projet here](https://github.com/jeshraghian/snntorch)